# Токенизация

В этом ноутбуке мы поговорим о том, что такое токенайзер, как его создать и какие интересные аспекты поведения моделей связаны с токенизацией.
Чтобы посмотреть, как работают разные токенизаторы, можно поиграть с этим [сайтом](https://tiktokenizer.vercel.app/).

Зачем вообще нужна токенизация? Это способ представить текст (или картинку) в виде достаточно небольших кусочков, которые можно передать как некоторые дискретные единицы в нейронную сеть или другую модель. В идеале, токенизатор должен разбивать текст так, чтобы каждый токен нес в себе больше информации, чем отдельный символ, но словарь возможных токенов был достаточно универсальным.

Существует множество разных токенизаторов, самый простой - просто разбить текст на отдельные символы.


In [ ]:
text = "Hi! Please help me to write 你好 in english!"

In [ ]:
print("Tokenized sequence: ", " ".join([str(ord(x)) for x in text]))
print(f"Number of letters: ", len([ord(x) for x in text]))

In [ ]:
tokens = text.encode("utf-8") # todo: try utf-16,utf-32
print("Tokenized sequence (raw): ", tokens)
# Выведите численные представления
print("Tokenized sequence: ", tokens)
print(f"Number of tokens: ", len(tokens))

Вопросы: 
- если мы будем использовать кодировку в utf-8, какого размера будет словарь? 
- В чем минус такой кодировки?
- какие еще способы токенизации вы знаете?


Вообще, токенизаторы не являются частью модели, но модель использует токены и никогда не видит сам текст:

![alt_text](../images/tokenizer.png)

Вопрос: что будет, если заменить токенищатор модели после  ее обучения?
Так как токенизатор независимая единица, он использует свой собственный обучающий датасет (который может отличаться от того, на котором претрейнилась модель). 

Вопросы: 
- Почему модели долго не умели считать буквы в словах?
- Почему моделям часто сложно делать простейшие арифметические операции?
- Почему "ранние" модели плозо говорили на русском или китайском?
- Почему YAML лучше JSON для LLM?
- Почему модель иногда вдруг начинает генерировать бред?


##  BPE алгоритм

Byte-Pair Encoding (BPE) это (по идее авторов) метод сжатия текстовых данных. Основная идея BPE заключается в итеративном объединении наиболее часто встречающихся пар символов (в текстовом корпусе) и формировании из них токенов, которые в последующем используются для токенизации.

### Алгоритм
1) Подготавливается словарь токенов (пока пустой)
2) Весь корпус разбивается на символы (байты). Уникальные символы добавляются в словарь токенов. Вопрос: почему их наджо сразу добавить в словарь?
3) В цикле (пока не будет достигнут желаемый размер словаря):
   1) Рассчитывается частот каждой соседней пары токенов в корпусе
   2) Наиболее частая пара объединяется в токен: добавляется в словарь, после чего объединяется в текстовом корпусе.

Базовый алгоритм обычно дополняется разными важными для разработчиков вещами:
- Появляется претокенизация (с помощью регулярок) для подготовки корпуса - он разбивается на чанки, что ускоряет BPE.
- В GPT-2 нельзя объединять в пару токены разных типов (знак препинания и букву, например).
- D GPT-3 нельзя объединять более 3 цифр подряд.
- В LLaMA нельзя создать токены только из пробелов или управляющих символов.

Кроме того, эффективность кодировки сильно зависит от корпуса, поэтому токенизаторы довольно сильно отличаются.

Попробуем реализовать свой собственный BL-BPE.


Для начала реализуем базовые функции - merge, get_stats, потом добавим их в более чистую версию класса. Обучать будем на игрушечном датасете:

In [ ]:
dataset = "Я в своем познании настолько преисполнился, что я как будто бы уже сто триллионов миллиардов лет проживаю на триллионах и триллионах таких же планет, как эта Земля, мне этот мир абсолютно понятен, и я здесь ищу только одного - покоя, умиротворения и вот этой гармонии, от слияния с бесконечно вечным, от созерцания великого фрактального подобия и от вот этого замечательного всеединства существа, бесконечно вечного, куда ни посмотри, хоть вглубь - бесконечно малое, хоть ввысь - бесконечное большое, понимаешь? А ты мне опять со своим вот этим, иди суетись дальше, это твоё распределение, это твой путь и твой горизонт познания и ощущения твоей природы, он несоизмеримо мелок по сравнению с моим, понимаешь? Я как будто бы уже давно глубокий старец, бессмертный, ну или там уже почти бессмертный, который на этой планете от её самого зарождения, ещё когда только Солнце только-только сформировалось как звезда, и вот это газопылевое облако, вот, после взрыва, Солнца, когда оно вспыхнуло, как звезда, начало формировать вот эти коацерваты, планеты, понимаешь, я на этой Земле уже как будто почти пять миллиардов лет живу и знаю её вдоль и поперёк этот весь мир, а ты мне какие-то... мне не важно на твои тачки, на твои яхты, на твои квартиры, там, на твоё благо. Я был на этой планете бесконечным множеством, и круче Цезаря, и круче Гитлера, и круче всех великих, понимаешь, был, а где-то был конченым говном, ещё хуже, чем здесь. Я множество этих состояний чувствую. Где-то я был больше подобен растению, где-то я больше был подобен птице, там, червю, где-то был просто сгусток камня, это всё есть душа, понимаешь? Она имеет грани подобия совершенно многообразные, бесконечное множество. Но тебе этого не понять, поэтому ты езжай себе , мы в этом мире как бы живем разными ощущениями и разными стремлениями, соответственно, разное наше и место, разное и наше распределение. Тебе я желаю все самые крутые тачки чтоб были у тебя, и все самые лучше самки, если мало идей, обращайся ко мне, я тебе на каждую твою идею предложу сотню триллионов, как всё делать. Ну а я всё, я иду как глубокий старец,узревший вечное, прикоснувшийся к Божественному, сам стал богоподобен и устремлен в это бесконечное, и который в умиротворении, покое, гармонии, благодати, в этом сокровенном блаженстве пребывает, вовлеченный во всё и во вся, понимаешь, вот и всё, в этом наша разница. Так что я иду любоваться мирозданием, а ты идёшь преисполняться в ГРАНЯХ каких-то, вот и вся разница, понимаешь, ты не зришь это вечное бесконечное, оно тебе не нужно. Ну зато ты, так сказать, более активен, как вот этот дятел долбящий, или муравей, который очень активен в своей стезе, поэтому давай, наши пути здесь, конечно, имеют грани подобия, потому что всё едино, но я-то тебя прекрасно понимаю, а вот ты меня - вряд ли, потому что я как бы тебя в себе содержу, всю твою природу, она составляет одну маленькую там песчиночку, от того что есть во мне, вот и всё, поэтому давай, ступай, езжай, а я пошел наслаждаться прекрасным осенним закатом на берегу теплой южной реки. Всё, ступай, и я пойду."

In [ ]:
tokens = dataset.encode("utf-8") # raw bytes
tokens = list(map(int, tokens))

In [ ]:
def get_stats(ids: list[int]) -> dict[tuple[int, int], int]:
    """counts pairs in text. Note that consecutive pairs overlap (if you have abc, pairs are ab. bc)"""
    counts = {}
    # collect number of token co-occurencies
    # good way is to use zip
    return counts

get_stats(dataset)


Раскодируем самые частые пары:

In [ ]:
# Your code: get sorted dict, try some pairs, convert to bytes, than use decode("utf-8")

Далее научимся мержить токены: 

In [ ]:
def merge(ids: list[int], pair: tuple[int, int], new_id: int) -> list[int] :
    """
    Merges all the occurencies of pair in list of ids int one new_id.
    """
    new_ids = []
    i = 0
    while i < len(ids):
        # check that ids[i] and [i+1] are both from pair and put the new id to list 
        # with results
        # else just add current id to it and go to the next token
    return new_ids

print(merge([1,2,3,4,5], (2,3), 9))
print(merge([1,2,3,4,5], (1,2), 9))
print(merge([1,2,3,4,5], (4,5), 9))
print(merge([1,2,3,4,5], (5,6), 9))
print(merge([], (5,6), 9))


Теперь нужно сделать прототип одной итерации алдгоритма:

In [ ]:
# get stats, merge, add new token to vocab

In [ ]:
len(tokens), len(tokens_new)

И собрать все воедино:

In [ ]:
tokens_orig = list(tokens)


In [ ]:
max_vocab_len = 100
num_merges = # calculate how much merges are allowed
first_merge_id = 256
merges = {}
for i in range(num_merges):
    # get best pair, merge it and add the merge to the merges dict (key is pair and value is new id)
    print(f"New merged pair is: {next_pair}")

Далее нам надо научиться кодировать и декодировать последовательности, спользуя наши мержи. Тут я перейду к классу, реализовывающему токенизатор:

In [ ]:
class BPE():
    def __init__(self, vocab_size: int):
        self.merges = {}
        self.special_tokens = {}
        self.vocab = self._build_vocab()
        self.vocab_size = vocab_size

    def _build_vocab(self):
        # дополнительный метод, который помогает построить словарь.
        # начальный словарь - содержит все возможные символы по одному
        vocab = {idx: bytes([idx]) for idx in range(256)} 
        # конкатенируем байты из наших мерджей
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]

        # не забываем про <bos>,<eos>, <|user|>, <think> и тд.
        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab

    def get_stats(self, ids: list[int]) -> dict[tuple[int, int], int]:
        """counts pairs in text. Note that consecutive pairs overlap (if you have abc, pairs are ab. bc)"""
        # copy code here 
        return counts

    def merge(self, ids: list[int], pair: tuple[int, int], new_id: int) -> list[int] :
        """
        Merges all the occurencies of pair in list of ids int one new_id.
        """
        # copy code here 
        return new_ids

    def fit(self, dataset: list[int], verbose: bool = True) -> None:
        # copy code here 
        for i in range(num_merges):
            # copy code here
            if verbose:
                print(f"New merged pair is: {next_pair} -> {first_merge_id + i}")
                
        self.vocab = self._build_vocab()

    def encode(self, text: str) -> list[int]:
        """ Get sequence of token ids using merge dict
        """
        # your code - there are several ways to encode sequence
        # you can reuse stats and merge code (obviously if there is more than two tokens)
        # after get of stats, you need to find first pair suitable for merges in merges dict:
        # min(stats, key=lambda x: self.merges.get(x, float("inf")))
        # In that case, if you used all possible merges from dict, return encoded sequence.
        
        return tokens


    def decode(self, token_ids: list[int]) -> str:
        """  
        From list of ids we get string. Use b"", decode("utf-8"). Use "replace" as param of decode.
        """
        return # your code 


In [ ]:
bpe = BPE(276)
bpe.fit(tokens_orig, verbose=True)

In [ ]:
len(bpe.vocab)

In [ ]:
# Try to decode something

In [ ]:
# Try to encode and decode back something

Мы сейчас построили самую простую версию BPE токенизатора, однако такой метод приводит к субоптимальному поведению и очень большим словарям. Поэтому многие компании расширяют его, оптимизируя некоторые части.
Например, в cтатье [GPT-2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) приводится, что наивная версия приводит к построению токенов из частых слов вместе со знаками препинания (dog? dog! dog.). Поэтому авторы решили добавить этап претокенизации - разбиения текста на чанки с помощью сложного regex паттерна и потом подсчету пар только внутри этих чанков.

Примеры паттернов:
- GPT2:  ```r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""```
- GPT4: ```r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""```
- Qwen-7B: ```r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""```

In [ ]:
import regex as re
pattern = # Compile pattern
print(re.findall(pattern, "I'm a happy small cat in the forrest."))

###  Разбор паттерна GPT-2 (от LLM-ки)

Паттерн `r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""` состоит из 5 альтернатив (разделены `|`), которые обрабатываются по порядку:

1. **`'(?:[sdmt]|ll|ve|re)`** - Апострофы с английскими сокращениями:
   - `'s`, `'d`, `'m`, `'t` (is, had, am, not)
   - `'ll`, `'ve`, `'re` (will, have, are)
   - Это позволяет отделить сокращения от слов

2. **` ?\p{L}+`** - Слова (буквы):
   - ` ?` - опциональный пробел перед словом
   - `\p{L}+` - одна или более букв (Unicode property для всех букв)
   - Примеры: "hello", "мир", "你好"

3. **` ?\p{N}+`** - Числа:
   - ` ?` - опциональный пробел перед числом
   - `\p{N}+` - одна или более цифр (Unicode property)
   - Примеры: "123", "42"

4. **` ?[^\s\p{L}\p{N}]+`** - Знаки препинания и специальные символы:
   - ` ?` - опциональный пробел перед
   - `[^\s\p{L}\p{N}]+` - один или более символов, которые НЕ пробелы, НЕ буквы, НЕ цифры
   - Примеры: "!", "?", ".", ",", "@#$"

5. **`\s+(?!\S)|\s+`** - Пробелы:
   - `\s+(?!\S)` - пробелы, за которыми НЕ следует непробельный символ (конец строки/слова)
   - `\s+` - любые пробелы (fallback)
   - Это позволяет отделить пробелы от слов

**Цель**: Разделить текст на логические чанки (слова, числа, знаки препинания), чтобы BPE не создавал токены типа "word!" или "word.", а работал только внутри этих чанков.


Чтобы кратенько посмотреть, как все это работает в реальных моделях (а также как обрабатываются спец-символы),  посмотрим на код [Qwen-7B](https://huggingface.co/Qwen/Qwen-7B).

### Использование tiktoken

Библиотека **tiktoken** от OpenAI позволяет использовать предобученные токенизаторы различных моделей от OpenAI. Как мы видели в коде Qwen, там внутри тоже используется эта библиотека.

Для быстрого примера  используем токенизатор, который используетися в GPT-4o.

In [ ]:
import tiktoken

# Загружаем токенизатор для Qwen-7B
# tiktoken автоматически скачает необходимые файлы при первом использовании
enc = tiktoken.get_encoding("o200k_base")

# Короткая строка для примера
text = "Привет! Hello! 你好! How are you?"

token_ids = enc.encode(text)
print(f"Original text: '{text}'")
print(f"tokens: {token_ids}")
print(f"number of tokens: {len(token_ids)}")

print(f"\ndecoded text '{enc.decode(token_ids)}'")

print(f"\ntokens as bytes:")
for i, token_id in enumerate(token_ids[:10]):  # показываем первые 10
    token_bytes = enc.decode_single_token_bytes(token_id)
    print(f"  Token {token_id}: {token_bytes}")

special_tokens = list(enc.special_tokens_set)
print(f" \nSpecial tokens: {special_tokens}")
# token_ids = enc.encode(special_tokens[0])
# print(f"tokens of first special token: {token_ids}")
token_ids = enc.encode(special_tokens[0], allowed_special={special_tokens[0]})
print(f"tokens of first special token: {token_ids}")

### Следующие шаги

Следующее, что имеет смысл сделать - закончить реализацию токенайзера, добавив в него специальные токены и regex-паттерны. Можно попробовать обучить токенайзер с конкретным regex-паттерном на большом наборе данных (попробуйте использовать какой-нибудь открытый датасет или несколько) и проверить, насколько ваш токенизатор пересекается с токенизатором выбранной модели.

Далее, по поведению модели относительно некооторых токенов можно понять, на чсем она обучалась, например, есть интересный [блок-пост](https://fi-le.net/oss/) и сопроводительный код к нему на [гитхабе](https://github.com/lennart-finke/gpt-oss/). Еще один пост: [про китайские токены в gpt-oss](https://www.technologyreview.com/2024/05/17/1092649/gpt-4o-chinese-token-polluted/). Попробуйте проанализиировать какую-нибудь модель - какие токены аномальны для нее? А, может, вы сможете проверить русские токены в токенизаторе?


## А что с изображениями:
Теперь поговорим о том, как можно представить в виде токенов картинку.  Vision Transformer (ViT) разбивает изображение на патчи (patches), которые можно рассматривать как токены изображения.


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from transformers import ViTImageProcessor, ViTModel
import requests
from io import BytesIO


In [ ]:
# Загружаем предобученный процессор и модель ViT из Hugging Face
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

# Получаем параметры из конфигурации модели
patch_size = model.config.patch_size  # размер патча (16 для patch16 моделей)
image_size = processor.size['height']  # размер изображения (224)

print(f"Размер патча: {patch_size}x{patch_size} пикселей")
print(f"Размер изображения: {image_size}x{image_size} пикселей")


## Vision Transformer (ViT)

**Vision Transformer (ViT)** - это трансформерная архитектурадля обработки изображений. Впервые предложена в статье ["An Image is Worth 16x16 Words"](https://arxiv.org/abs/2010.11929) в 2020 году.
Основная идея заключается в следующем:
1) Изображение разбивается на небольшие квадратные патчи, которые рассматриваются как "токены" изображения.
2) Каждый патч сверткой вытягивается в одномерный вектор и проходит через линейный слой (embedding layer), превращаясь в эмбеддинг фиксированной размерности.
3) **[CLS] токен**: В начало последовательности добавляется специальный токен [CLS] (classification token), который после прохождения через модель содержит информацию обо всем изображении и используется для классификации (подробнее на лекциях и практике по BERT).
4) Последовательность эмбеддингов патчей проходит через стандартный Transformer Encoder (подробнее на лекциях и практике по BERT).


In [ ]:

image = Image.open("..\images\kot_mjaukaet.jpg").convert('RGB')
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title('Исходное изображение')
plt.show()


In [ ]:

inputs = processor(images=image, return_tensors="pt")

patch_size = model.config.patch_size  # размер патча (16 для patch16 моделей)
image_size = processor.size['height']  # размер изображения (224)

num_patches_per_side = image_size // patch_size
total_patches = num_patches_per_side ** 2

print(f"Изображение разбито на {num_patches_per_side}x{num_patches_per_side} = {total_patches} патчей")
print(f"Размер каждого патча: {patch_size}x{patch_size} пикселей")


In [ ]:
def visualize_patches(image, patch_size=16, num_patches_per_side=14):
    img_array = np.array(image.resize((224, 224)))
    fig, axes = plt.subplots(
        num_patches_per_side, num_patches_per_side, figsize=(20, 20)
    )
    fig.suptitle('Токены изображения (патчи 16x16)', fontsize=16)
    for i in range(num_patches_per_side):
        for j in range(num_patches_per_side):
            y_start = i * patch_size
            y_end = y_start + patch_size
            x_start = j * patch_size
            x_end = x_start + patch_size
            
            patch = img_array[y_start:y_end, x_start:x_end]
            
            axes[i, j].imshow(patch)
            axes[i, j].axis('off')
            axes[i, j].set_title(f'({i},{j})', fontsize=6)
    
    plt.tight_layout()
    plt.show()
    return img_array

patched_image = visualize_patches(image, patch_size=patch_size, num_patches_per_side=num_patches_per_side)
